In [3]:
!pip install tensorflow-addons
import pandas as pd
import tensorflow as tf
import numpy as np
from gensim.utils import simple_preprocess
import time
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.python.keras.preprocessing.text import Tokenizer
import os
import tensorflow_addons as tfa
from gensim.models import FastText, Word2Vec
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers import MaxPooling1D, Dense, Flatten
from keras.initializers import Constant
import keras.layers as L
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, GRU, Input, Flatten, Bidirectional, SimpleRNN, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import accuracy_score, \
                            precision_score, recall_score, \
                            f1_score

def rnn_model(num_words, EMBEDDING_DIM, max_length, embedding_matrix):
  model = Sequential() 
  model.add(Embedding(num_words,
                      EMBEDDING_DIM,
                      input_length=max_length,
                      trainable=False,
                      embeddings_initializer=Constant(embedding_matrix)))
 
  model.add(SimpleRNN(128, return_sequences=True, dropout=0.2))
  model.add(SimpleRNN(128, return_sequences=True, dropout=0.2))
  model.add(SimpleRNN(128, return_sequences=True, dropout=0.2))
  model.add(SimpleRNN(128, dropout=0.2))
  model.add(Dense(1,activation='sigmoid'))
  return model

def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print("================================================================================================")
    print(len(gloveModel)," words loaded!")
    print("================================================================================================")
    return gloveModel

!git clone https://github.com/MaazAmjad/Urdu-Fake-news-detection-FIRE2021 UrduFake
!unzip -o "/content/UrduFake/Test Dataset @ FIRE 2021".zip 
!unzip -o "/content/UrduFake/Training Dataset@FIRE2021".zip

datasets = {
      "Train": "/content/Training Dataset@FIRE2021/Train",
      "Validation": "/content/Training Dataset@FIRE2021/Test",
      "Test": "/content/Test Dataset"
}
for dataset in datasets.keys():
  if dataset == 'Test':
    df = pd.DataFrame(columns=['Text', 'File No'])
  else:
    df = pd.DataFrame(columns=['Text', 'Label'])
  fileList = []
  for root, dirs, files in os.walk(datasets[dataset]):
    for file in files:
      if dataset == 'Test':
        classLabel = file.replace('.txt', '')
      else:
        classLabel = os.path.join(root,file).split('/')[-2]
      fileList.append({'path':os.path.join(root,file),'class':classLabel})

  for _file in fileList:
      fileRead = open(_file['path'], mode='r', encoding="UTF-8").read()
      if dataset == 'Test':
        df = df.append({'Text':fileRead, 'File No':_file['class']}, ignore_index = True)
      else:
        df = df.append({'Text':fileRead, 'Label':_file['class']}, ignore_index = True)

  df.to_csv(dataset+'.csv', index=False) 

dfTrain = pd.read_csv('Train.csv')
dfValid = pd.read_csv('Validation.csv')
dfTest = pd.read_csv('Test.csv')

dataFrames = [dfTrain, dfValid, dfTest]
for dataFrame in dataFrames:
  dataFrame['Text'].replace(np.NaN, 'Null',inplace=True)
  dataFrame['Text'] = dataFrame['Text'].apply(lambda x:str(x))
  if 'Label' in dataFrame.columns: 
    dataFrame['Label'].replace(np.NaN, 0,inplace=True)
    dataFrame['Label'] = np.where(dataFrame['Label']== 'Fake', 1, 0)

dfTrain['tokenized_text'] = [simple_preprocess(line, deacc=False,min_len=1, max_len=1000) for line in dfTrain['Text']]

"""""""""""""""""""""""""""""""""""""""""""GloveText Embedding Creation"""""""""""""""""""""""""""""""""""""""
texts = ""
for text in dfTrain['tokenized_text'].values:
  texts = texts + " " + " ".join(text)

file = open('/content/text.txt', 'w+')
file.write(texts)

!git clone https://github.com/stanfordnlp/glove
!cd glove && make

window = 3
size = 100
min_count = 1 


print("=========================================================")
!/content/glove/build/vocab_count -verbose 2 -min-count 1 < /content/text.txt > /content/vocab.txt
!/content/glove/build/cooccur -verbose 2 -symmetric 1 -window-size {window} -vocab-file /content/vocab.txt -overflow-file tempoverflow < /content/text.txt > /content/cooccurrences.bin

sg_model_name = "GloveText_" + str(size) + "_" + str(window) + "_" + str(min_count) + "_SkipGram" 

!/content/glove/build/shuffle -verbose 2  < /content/cooccurrences.bin > /content/cooccurrences.shuf.bin
!/content/glove/build/glove -input-file /content/cooccurrences.shuf.bin -vocab-file /content/vocab.txt -save-file {sg_model_name} -gradsq-file /content/gradsq -verbose 2 -vector-size {size}

sg_model = loadGloveModel(sg_model_name+".txt")

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(dfTrain['Text'].values)
sequences = tokenizer_obj.texts_to_sequences(dfTrain['Text'].values)
word_index = tokenizer_obj.word_index
max_length = max([len(str(s).split()) for s in dfTrain['Text'].values])
review_pad = pad_sequences(sequences,maxlen=max_length)

tokenizer_obj1 = Tokenizer()
tokenizer_obj1.fit_on_texts(dfValid['Text'].values)
sequences1 = tokenizer_obj1.texts_to_sequences(dfValid['Text'].values)
review_pad1 = pad_sequences(sequences1,maxlen=max_length)

tokenizer_obj2 = Tokenizer()
tokenizer_obj2.fit_on_texts(dfTest['Text'].values)
sequences2 = tokenizer_obj2.texts_to_sequences(dfTest['Text'].values)
review_pad2 = pad_sequences(sequences2,maxlen=max_length)


labelsTrain = dfTrain['Label'].values
labelsValid = dfValid['Label'].values

EMBEDDING_DIM = int(size)
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))
for word,i in word_index.items():
    if i>num_words:
        continue
    if word in sg_model:
        embedding_vector=sg_model[word]
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector


model = rnn_model(num_words, EMBEDDING_DIM, max_length, embedding_matrix)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('/content/best_model.h5', monitor='val_acc', mode='max', min_delta=1, save_best_only=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
with tf.device('/device:GPU:0'):
    model.fit(review_pad, labelsTrain, batch_size=100, epochs=50, validation_data=(review_pad1, labelsValid), verbose=2, shuffle=True, callbacks=[es,mc])
    predict_model = load_model('/content/best_model.h5')
    y_pred = (np.asarray(predict_model.predict(review_pad1))).round()
    y = labelsValid
    Accuracy = accuracy_score(y, y_pred)
    Precision = precision_score(y, y_pred, zero_division=1)
    Recall =  recall_score(y, y_pred, zero_division=1)
    f1Score = ((Precision*Recall)/(Precision+Recall))*2
    print("=============================================")
    print(f"Accuracy for {dataset} Dataset:{Accuracy}")
    print(f"Precision for {dataset} Dataset:{Precision}")
    print(f"Recall for {dataset} Dataset:{Recall}")
    print(f"F1 Score for {dataset} Dataset:{f1Score}")

    y_pred = (np.asarray(predict_model.predict(review_pad2))).round()
    dfTest['Label'] =  np.where(y_pred== 1, 'F', 'R')
    del(dfTest['Text'])
    dfTest.to_csv('Submission.csv', index=False)

fatal: destination path 'UrduFake' already exists and is not an empty directory.
Archive:  /content/UrduFake/Test Dataset @ FIRE 2021.zip
  inflating: Test Dataset/1.txt      
  inflating: Test Dataset/10.txt     
  inflating: Test Dataset/100.txt    
  inflating: Test Dataset/101.txt    
  inflating: Test Dataset/102.txt    
  inflating: Test Dataset/103.txt    
  inflating: Test Dataset/104.txt    
  inflating: Test Dataset/105.txt    
  inflating: Test Dataset/106.txt    
  inflating: Test Dataset/107.txt    
  inflating: Test Dataset/108.txt    
  inflating: Test Dataset/109.txt    
  inflating: Test Dataset/11.txt     
  inflating: Test Dataset/110.txt    
  inflating: Test Dataset/111.txt    
  inflating: Test Dataset/112.txt    
  inflating: Test Dataset/113.txt    
  inflating: Test Dataset/114.txt    
  inflating: Test Dataset/115.txt    
  inflating: Test Dataset/116.txt    
  inflating: Test Dataset/117.txt    
  inflating: Test Dataset/118.txt    
  inflating: Test Dataset/